In [41]:
"""
https://leetcode.com/problems/meeting-scheduler/
https://leetcode.ca/2019-04-12-1229-Meeting-Scheduler/

Given the availability time slots arrays slots1 and slots2 of two people and a meeting duration duration, 
return the earliest time slot that works for both of them and is of duration duration.

If there is no common time slot that satisfies the requirements, return an empty array.

The format of a time slot is an array of two elements [start, end] representing an inclusive time range from start to end.

It is guaranteed that no two availability slots of the same person intersect with each other. 
That is, for any two time slots [start1, end1] and [start2, end2] of the same person, either start1 > end2 or start2 > end1.

 

Constraints:
1 <= slots1.length, slots2.length <= 10^4
slots1[i].length, slots2[i].length == 2
slots1[i][0] < slots1[i][1]
slots2[i][0] < slots2[i][1]
0 <= slots1[i][j], slots2[i][j] <= 10^9
1 <= duration <= 10^6
"""
def meetingScheduler(slots1, slots2, duration):
    i1, i2 = 0, 0

    while i1 < len(slots1) and i2 < len(slots2):
        # """ three cases
        #     []
        #         []
        #     [   ]
        #       [    ]
        #     [         ]
        #        [   ]
        # """
        start = max(slots1[i1][0],
                    slots2[i2][0])
        end   = min(slots1[i1][1],
                    slots2[i2][1])

        if end-start >= duration:
            return [start, start+duration]

        # drop meeting slot that ends earlier
        if slots1[i1][1] <= slots2[i2][1]:
            i1+=1
        else:
            i2+=1

    return []



tests = [
    ([[10,50],[60,120],[140,210]], [[0,15],[60,70]], 8, [60,68]),
    ([[10,50],[60,120],[140,210]], [[0,15],[60,70]], 12, []),
] 
for t in tests:
    retVal = meetingScheduler(t[0], t[1], t[2])
    print(t, retVal)
    assert(retVal == t[3])

([[10, 50], [60, 120], [140, 210]], [[0, 15], [60, 70]], 8, [60, 68]) [60, 68]
([[10, 50], [60, 120], [140, 210]], [[0, 15], [60, 70]], 12, []) []


In [36]:
"""
https://leetcode.com/problems/maximum-profit-in-job-scheduling/

We have n jobs, where every job is scheduled to be done from startTime[i] to endTime[i], obtaining a profit of profit[i].

You're given the startTime, endTime and profit arrays, return the maximum profit you can take such that
there are no two jobs in the subset with overlapping time range.

If you choose a job that ends at time X you will be able to start another job that starts at time X.

"""


"""
greedy: not optimal (counter example: 5 start to end, two jobs (4+3) within the same time period
    sort jobs by descending order of profit
    take most profiable first
    then go down the list and perform job if scheduling allows


permute all combinations: create all possible combinations of jobs

    sort jobs by start time: (startT, endT, profit)
        so that the next possible job can be found fast (the next job start time should be on or after the endT of the current job)

    for every job, starting from the earliest starting job, start adding/not adding available jobs


"""
def getMaxProfit(startTs, endTs, profits):

    jobs = sorted(zip(startTs, endTs, profits))
    numJobs = len(jobs)

    maxProfit = 0
    performed = set()
    profit = 0
    def _add(jId, startTime):
        nonlocal maxProfit, profit
        if maxProfit < profit:
            maxProfit = profit

        if jId  == len(jobs):
            return

        # take job
        if jId not in performed and startTime <= jobs[jId][0]:
            performed.add(jId)
            profit += jobs[jId][2]
            _add(jId+1, jobs[jId][1])
            performed.remove(jId)
            profit -= jobs[jId][2]
        # do not take job
        _add(jId+1, startTime)

    _add(0, jobs[0][0])

    return maxProfit
        

tests = [
    ([1,2,3,3], [3,4,5,6], [50,10,40,70], 120),
    ([1,2,3,4,6], [3,5,10,6,9], [20,20,100,70,60], 150),
    ([1,1,1], [2,3,4], [5,6,4], 6)
]
for t in tests:
    retVal = getMaxProfit(t[0], t[1], t[2])
    print(t, retVal)
    assert(retVal == t[3])

([1, 2, 3, 3], [3, 4, 5, 6], [50, 10, 40, 70], 120) 120
([1, 2, 3, 4, 6], [3, 5, 10, 6, 9], [20, 20, 100, 70, 60], 150) 150
([1, 1, 1], [2, 3, 4], [5, 6, 4], 6) 6


In [26]:
"""
https://leetcode.com/problems/traffic-light-controlled-intersection/
https://leetcode.ca/2019-06-01-1279-Traffic-Light-Controlled-Intersection/


There is an intersection of two roads.
First road is road A where cars travel from North to South in direction 1 and from South to North in direction 2.
Second road is road B where cars travel from West to East in direction 3 and from East to West in direction 4.

dir 1   A A A
  |     A A A  <- dir 4
  V     A A A
B B B B       B B B B 
B B B B       B B B B 
B B B B       B B B B 
        A A A    ^
  ->    A A A    |
 dir 3  A A A  dir 2

There is a traffic light located on each road before the intersection.
A traffic light can either be green or red.

Green means cars can cross the intersection in both directions of the road.
Red means cars in both directions cannot cross the intersection and must wait until the light turns green.
The traffic lights cannot be green on both roads at the same time. 
That means when the light is green on road A, it is red on road B and when the light is green on road B, it is red on road A.

Initially, the traffic light is green on road A and red on road B. 
When the light is green on one road, all cars can cross the intersection in both directions until the light becomes green on the other road. 
No two cars traveling on different roads should cross at the same time.

Design a deadlock-free traffic light controlled system at this intersection.

Implement the function void carArrived(carId, roadId, direction, turnGreen, crossCar) where:

carId is the id of the car that arrived.
roadId is the id of the road that the car travels on.
direction is the direction of the car.
turnGreen is a function you can call to turn the traffic light to green on the current road.
crossCar is a function you can call to let the current car cross the intersection.
Your answer is considered correct if it avoids cars deadlock in the intersection. 
Turning the light green on a road when it was already green is considered a wrong answer.


Constraints:

1 <= cars.length <= 20
cars.length = directions.length
cars.length = arrivalTimes.length
All values of cars are unique
1 <= directions[i] <= 4
arrivalTimes is non-decreasing
"""
import threading
import time

class TrafficLight:
    def __init__(self):
        self.lock = threading.Lock()
        self.road = 1 # 1 (dir 1, 2) or 2 (dir 3, 4)
    
    def carArrived(self, carId, roadId, direction, turnGreen, crossCar):
        self.lock.acquire()
        if self.road != roadId:
            self.road = roadId
            turnGreen()
        crossCar()
        self.lock.release()


def test(cars, directions, arrivalTimes):
    light = TrafficLight()
    idx = 0
    threads = []
    for t in range(1, arrivalTimes[-1]+1):
        time.sleep(0.5)
        while idx < len(arrivalTimes) and t >= arrivalTimes[idx]:
            carId = cars[idx]
            roadId = (directions[idx]-1)//2+1
            direction = directions[idx]
            turnGreen = lambda: print("Traffic Light On Road", "A" if roadId == 1 else "B", "Is Green")
            crossCar = lambda: print("Car", carId, "Has Passed Road", "A" if roadId == 1 else "B", "In Direction", direction)
            car = threading.Thread(target=light.carArrived, args=[carId,
                                                                roadId,
                                                                direction,
                                                                turnGreen,
                                                                crossCar])
            car.start()
            threads.append(car)
            idx += 1

    for car in threads:
        car.join()
    print()

test([1,3,5,2,4], [2,1,2,4,3], [10,20,30,40,50])
# "Car 1 Has Passed Road A In Direction 2",    #// Traffic light on road A is green, car 1 can cross the intersection.
# "Car 3 Has Passed Road A In Direction 1",    #// Car 3 crosses the intersection as the light is still green.
# "Car 5 Has Passed Road A In Direction 2",    #// Car 5 crosses the intersection as the light is still green.
# "Traffic Light On Road B Is Green",          #// Car 2 requests green light for road B.
# "Car 2 Has Passed Road B In Direction 4",    #// Car 2 crosses as the light is green on road B now.
# "Car 4 Has Passed Road B In Direction 3"     #// Car 4 crosses the intersection as the light is still green.

test([1,2,3,4,5], [2,4,3,3,1], [10,20,30,40,40])
# "Car 1 Has Passed Road A In Direction 2",    #// Traffic light on road A is green, car 1 can cross the intersection.
# "Traffic Light On Road B Is Green",          #// Car 2 requests green light for road B.
# "Car 2 Has Passed Road B In Direction 4",    #// Car 2 crosses as the light is green on road B now.
# "Car 3 Has Passed Road B In Direction 3",    #// Car 3 crosses as the light is green on road B now.
# "Traffic Light On Road A Is Green",          #// Car 5 requests green light for road A.
# "Car 5 Has Passed Road A In Direction 1",    #// Car 5 crosses as the light is green on road A now.
# "Traffic Light On Road B Is Green",          #// Car 4 requests green light for road B. 
#                                              #   Car 4 blocked until car 5 crosses and then traffic light is green on road B.
# "Car 4 Has Passed Road B In Direction 3"     #// Car 4 crosses as the light is green on road B now.
#
#This is a dead-lock free scenario. 
# Note that the scenario when car 4 crosses before turning light into green on road A and allowing car 5 to pass is also correct 
# and Accepted scenario.

Car 1 Has Passed Road A In Direction 2
Car 3 Has Passed Road A In Direction 1
Car 5 Has Passed Road A In Direction 2
Traffic Light On Road B Is Green
Car 2 Has Passed Road B In Direction 4
Car 4 Has Passed Road B In Direction 3

Car 1 Has Passed Road A In Direction 2
Traffic Light On Road B Is Green
Car 2 Has Passed Road B In Direction 4
Car 3 Has Passed Road B In Direction 3
Car 4 Has Passed Road B In Direction 3
Traffic Light On Road A Is Green
Car 5 Has Passed Road A In Direction 1



In [ ]:
"""
https://leetcode.com/problems/students-and-examinations/

Table: Students

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| student_id    | int     |
| student_name  | varchar |
+---------------+---------+
student_id is the primary key (column with unique values) for this table.
Each row of this table contains the ID and the name of one student in the school.
 

Table: Subjects

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| subject_name | varchar |
+--------------+---------+
subject_name is the primary key (column with unique values) for this table.
Each row of this table contains the name of one subject in the school.
 

Table: Examinations

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| student_id   | int     |
| subject_name | varchar |
+--------------+---------+
There is no primary key (column with unique values) for this table. It may contain duplicates.
Each student from the Students table takes every course from the Subjects table.
Each row of this table indicates that a student with ID student_id attended the exam of subject_name.
 

Write a solution to find the number of times each student attended each exam.

Return the result table ordered by student_id and subject_name.

The result format is in the following example.

 

Example 1:

Input: 
Students table:
+------------+--------------+
| student_id | student_name |
+------------+--------------+
| 1          | Alice        |
| 2          | Bob          |
| 13         | John         |
| 6          | Alex         |
+------------+--------------+
Subjects table:
+--------------+
| subject_name |
+--------------+
| Math         |
| Physics      |
| Programming  |
+--------------+
Examinations table:
+------------+--------------+
| student_id | subject_name |
+------------+--------------+
| 1          | Math         |
| 1          | Physics      |
| 1          | Programming  |
| 2          | Programming  |
| 1          | Physics      |
| 1          | Math         |
| 13         | Math         |
| 13         | Programming  |
| 13         | Physics      |
| 2          | Math         |
| 1          | Math         |
+------------+--------------+
Output: 
+------------+--------------+--------------+----------------+
| student_id | student_name | subject_name | attended_exams |
+------------+--------------+--------------+----------------+
| 1          | Alice        | Math         | 3              |
| 1          | Alice        | Physics      | 2              |
| 1          | Alice        | Programming  | 1              |
| 2          | Bob          | Math         | 1              |
| 2          | Bob          | Physics      | 0              |
| 2          | Bob          | Programming  | 1              |
| 6          | Alex         | Math         | 0              |
| 6          | Alex         | Physics      | 0              |
| 6          | Alex         | Programming  | 0              |
| 13         | John         | Math         | 1              |
| 13         | John         | Physics      | 1              |
| 13         | John         | Programming  | 1              |
+------------+--------------+--------------+----------------+
Explanation: 
The result table should contain all students and all subjects.
Alice attended the Math exam 3 times, the Physics exam 2 times, and the Programming exam 1 time.
Bob attended the Math exam 1 time, the Programming exam 1 time, and did not attend the Physics exam.
Alex did not attend any exams.
John attended the Math exam 1 time, the Physics exam 1 time, and the Programming exam 1 time.
"""


In [25]:
"""
https://leetcode.com/problems/maximum-number-of-events-that-can-be-attended/

You are given an array of events where events[i] = [startDayi, endDayi]. Every event i starts at startDayi and ends at endDayi.

You can attend an event i at any day d where startTimei <= d <= endTimei. You can only attend one event at any time d.

Return the maximum number of events you can attend.

  [            ]
    []
      [   ]
         [ ]



Constraints:

1 <= events.length <= 10^5
events[i].length == 2
1 <= startDayi <= endDayi <= 10^5
"""

def maxAttend(events):
    _events = sorted(events)
    import heapq
    minheap = []
    startTime = _events[0][0]
    endTime = -1
    for s,e in _events:
        if e > endTime:
            endTime = e

    numAttended = 0
    for t in range(startTime, endTime+1):
        while len(minheap) > 0 and minheap[0][1][1] < t:
            heapq.heappop()

        for s,e in _events:
            if s < t:
                continue
            elif s == t:
                heapq.heappush(minheap, (e, (s,e)))
            elif s > t:
                break

        if len(minheap) > 0:
            _, (s, e) = heapq.heappop(minheap)
            numAttended += 1

    return numAttended



tests = [
    ([[1,2],[2,3],[3,4]], 3),
    ([[1,2],[2,3],[3,4],[1,2]], 4)
]
for t in tests:
    retVal = maxAttend(t[0])
    print(t, retVal)
    assert(retVal == t[1])

([[1, 2], [2, 3], [3, 4]], 3) 3
([[1, 2], [2, 3], [3, 4], [1, 2]], 4) 4


In [ ]:
"""
https://leetcode.com/problems/number-of-trusted-contacts-of-a-customer/


Table: Customers

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| customer_id   | int     |
| customer_name | varchar |
| email         | varchar |
+---------------+---------+
customer_id is the column of unique values for this table.
Each row of this table contains the name and the email of a customer of an online shop.
 

Table: Contacts

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | id      |
| contact_name  | varchar |
| contact_email | varchar |
+---------------+---------+
(user_id, contact_email) is the primary key (combination of columns with unique values) for this table.
Each row of this table contains the name and email of one contact of customer with user_id.
This table contains information about people each customer trust. The contact may or may not exist in the Customers table.
 

Table: Invoices

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| invoice_id   | int     |
| price        | int     |
| user_id      | int     |
+--------------+---------+
invoice_id is the column of unique values for this table.
Each row of this table indicates that user_id has an invoice with invoice_id and a price.
 

Write a solution to find the following for each invoice_id:

customer_name: The name of the customer the invoice is related to.
price: The price of the invoice.
contacts_cnt: The number of contacts related to the customer.
trusted_contacts_cnt: The number of contacts related to the customer and at the same time they are customers to the shop. (i.e their email exists in the Customers table.)
Return the result table ordered by invoice_id.

The result format is in the following example.

 

Example 1:

Input: 
Customers table:
+-------------+---------------+--------------------+
| customer_id | customer_name | email              |
+-------------+---------------+--------------------+
| 1           | Alice         | alice@leetcode.com |
| 2           | Bob           | bob@leetcode.com   |
| 13          | John          | john@leetcode.com  |
| 6           | Alex          | alex@leetcode.com  |
+-------------+---------------+--------------------+
Contacts table:
+-------------+--------------+--------------------+
| user_id     | contact_name | contact_email      |
+-------------+--------------+--------------------+
| 1           | Bob          | bob@leetcode.com   |
| 1           | John         | john@leetcode.com  |
| 1           | Jal          | jal@leetcode.com   |
| 2           | Omar         | omar@leetcode.com  |
| 2           | Meir         | meir@leetcode.com  |
| 6           | Alice        | alice@leetcode.com |
+-------------+--------------+--------------------+
Invoices table:
+------------+-------+---------+
| invoice_id | price | user_id |
+------------+-------+---------+
| 77         | 100   | 1       |
| 88         | 200   | 1       |
| 99         | 300   | 2       |
| 66         | 400   | 2       |
| 55         | 500   | 13      |
| 44         | 60    | 6       |
+------------+-------+---------+
Output: 
+------------+---------------+-------+--------------+----------------------+
| invoice_id | customer_name | price | contacts_cnt | trusted_contacts_cnt |
+------------+---------------+-------+--------------+----------------------+
| 44         | Alex          | 60    | 1            | 1                    |
| 55         | John          | 500   | 0            | 0                    |
| 66         | Bob           | 400   | 2            | 0                    |
| 77         | Alice         | 100   | 3            | 2                    |
| 88         | Alice         | 200   | 3            | 2                    |
| 99         | Bob           | 300   | 2            | 0                    |
+------------+---------------+-------+--------------+----------------------+
Explanation: 
Alice has three contacts, two of them are trusted contacts (Bob and John).
Bob has two contacts, none of them is a trusted contact.
Alex has one contact and it is a trusted contact (Alice).
John doesn't have any contacts.
"""

In [20]:
"""
https://leetcode.com/problems/minimum-number-of-frogs-croaking/

You are given the string croakOfFrogs, which represents a combination of the string "croak" from different frogs, 
that is, multiple frogs can croak at the same time, so multiple "croak" are mixed.

Return the minimum number of different frogs to finish all the croaks in the given string.

A valid "croak" means a frog is printing five letters 'c', 'r', 'o', 'a', and 'k' sequentially. 
The frogs have to print all five letters to finish a croak. If the given string is not a combination of a valid "croak" return -1.


Constraints:
1 <= croakOfFrogs.length <= 10^5
croakOfFrogs is either 'c', 'r', 'o', 'a', or 'k'.
"""

from collections import defaultdict

def minCroaking(s):
    counter = defaultdict(int)

    minNumFrogs = 0
    numFrogs = 0

    def _FrogsFinished():
        nonlocal numFrogs
        while counter["c"] > 0 and \
              counter["r"] > 0 and \
              counter["o"] > 0 and \
              counter["a"] > 0 and \
              counter["k"] > 0:
            counter["c"] -= 1
            counter["r"] -= 1
            counter["o"] -= 1
            counter["a"] -= 1
            counter["k"] -= 1
            numFrogs = max(0, numFrogs-1)

    for c in s:
        _FrogsFinished()
        counter[c] += 1
        numFrogs = max( counter["c"],
                        counter["r"],
                        counter["o"],
                        counter["a"],
                        counter["k"])
        if minNumFrogs < numFrogs:
            minNumFrogs = numFrogs

    _FrogsFinished()
    
    if sum( [ v for  k,v in counter.items() ] ) > 0:
        return -1

    return minNumFrogs


tests = [
    ("croakcroak", 1),
    ("crcoakroak", 2),
    ("croakcrook", -1)
]
for t in tests:
    retVal = minCroaking(t[0])
    print(t, retVal)
    assert(retVal == t[1])

('croakcroak', 1) 1
('crcoakroak', 2) 2
('croakcrook', -1) -1


In [15]:
"""
https://leetcode.com/problems/first-unique-number/
https://leetcode.ca/2019-10-29-1429-First-Unique-Number/


You have a queue of integers, you need to retrieve the first unique integer in the queue.

Implement the FirstUnique class:
FirstUnique(int[] nums) Initializes the object with the numbers in the queue.
int showFirstUnique() returns the value of the first unique integer of the queue, and returns -1 if there is no such integer.
void add(int value) insert value to the queue.
 

Constraints:

1 <= nums.length <= 10^5
1 <= nums[i] <= 10^8
1 <= value <= 10^8
At most 50000 calls will be made to showFirstUnique and add.
"""

from collections import defaultdict, deque

class FirstUnique:
    def __init__(self, nums):
        self.counter = defaultdict(int)
        self.queue = deque()
        for n in nums:
            self.add(n)
        
    def add(self, n):
        self.counter[n]+=1
        self.queue.append(n)
        while len(self.queue) > 0 and self.counter[self.queue[0]] > 1:
            self.queue.popleft()

    def showFirstUnique(self):
        if len(self.queue) == 0:
            return -1
        return self.queue[0]
        

firstUnique = FirstUnique([2,3,5])
assert(firstUnique.showFirstUnique() == 2)  # // return 2
firstUnique.add(5)                          # // the queue is now [2,3,5,5]
assert(firstUnique.showFirstUnique() == 2) # // return 2
firstUnique.add(2)                          # // the queue is now [2,3,5,5,2]
assert(firstUnique.showFirstUnique() == 3) # // return 3
firstUnique.add(3)                          # // the queue is now [2,3,5,5,2,3]
assert(firstUnique.showFirstUnique() == -1) # // return -1


firstUnique = FirstUnique([7,7,7,7,7,7])
assert(firstUnique.showFirstUnique() == -1)       # // return -1
firstUnique.add(7)                                # // the queue is now [7,7,7,7,7,7,7]
firstUnique.add(3)                                # // the queue is now [7,7,7,7,7,7,7,3]
firstUnique.add(3)                                # // the queue is now [7,7,7,7,7,7,7,3,3]
firstUnique.add(7)                                # // the queue is now [7,7,7,7,7,7,7,3,3,7]
firstUnique.add(17)                               # // the queue is now [7,7,7,7,7,7,7,3,3,7,17]
assert(firstUnique.showFirstUnique() == 17)       # // return 17


firstUnique = FirstUnique([809])
assert(firstUnique.showFirstUnique() == 809) # // return 809
firstUnique.add(809)                         # // the queue is now [809,809]
assert(firstUnique.showFirstUnique() == -1)  # // return -1
 

In [13]:
"""
https://leetcode.com/problems/shuffle-the-array/

Given the array nums consisting of 2n elements in the form [x1,x2,...,xn,y1,y2,...,yn].
Return the array in the form [x1,y1,x2,y2,...,xn,yn].

Constraints:
1 <= n <= 500
nums.length == 2n
1 <= nums[i] <= 10^3
"""

def shuffle(arr, n):
    """
    bubble down latter half (indices n down to 1 ~ 2n-1) n ~ 1 times?
    n 
        n times
    n+1
        n-1 times
    ..
    2n-1
        1 time

    0 1 2 3 4 5 6 7 8 910111213 
    a b c d e f g 1 2 3 4 5 6 7
    a b c d e f 1 g 2 3 4 5 6 7

    a 1 b c d e f g 2 3 4 5 6 7
    a 1 b 2 c d e f g 3 4 5 6 7
    """

    def _swap(i, j):
        tmp = arr[i]
        arr[i] = arr[j]
        arr[j] = tmp

    for i in range(n, 2*n):
        for j in range(2*n - i - 1):
            k = i - j
            _swap(k-1, k)

    return arr
    

tests = [
    ([2,5,1,3,4,7], 3, [2,3,5,4,1,7]),
    # x1=2, x2=5, x3=1, y1=3, y2=4, y3=7 then the answer is [2,3,5,4,1,7].
    ([1,2,3,4,4,3,2,1], 4, [1,4,2,3,3,2,4,1]),
    ([1,1,2,2], 2, [1,2,1,2])
]
for t in tests:
    retVal = shuffle(t[0], t[1])
    print(t, retVal)
    assert(retVal == t[2]) 

([2, 3, 5, 4, 1, 7], 3, [2, 3, 5, 4, 1, 7]) [2, 3, 5, 4, 1, 7]
([1, 4, 2, 3, 3, 2, 4, 1], 4, [1, 4, 2, 3, 3, 2, 4, 1]) [1, 4, 2, 3, 3, 2, 4, 1]
([1, 2, 1, 2], 2, [1, 2, 1, 2]) [1, 2, 1, 2]


In [5]:
"""
https://leetcode.com/problems/design-browser-history/

You have a browser of one tab where you start on the homepage and you can visit another url, 
get back in the history number of steps or move forward in the history number of steps.

Implement the BrowserHistory class:

BrowserHistory(string homepage) Initializes the object with the homepage of the browser.
void visit(string url) Visits url from the current page. It clears up all the forward history.
string back(int steps) Move steps back in history. 
If you can only return x steps in the history and steps > x, you will return only x steps. 
Return the current url after moving back in history at most steps.
string forward(int steps) Move steps forward in history. 
If you can only forward x steps in the history and steps > x, you will forward only x steps. 
Return the current url after forwarding in history at most steps.


Constraints:

1 <= homepage.length <= 20
1 <= url.length <= 20
1 <= steps <= 100
homepage and url consist of  '.' or lower case English letters.
At most 5000 calls will be made to visit, back, and forward.
"""


class BrowserHistory:
    """
        sol1: doubly-linked list
        sol2: array with position pointer
    """

    def __init__(self, homepage):
        self.history = [homepage]
        self.position = 0

    def visit(self, site):
        if self.position < len(self.history)-1:
            self.history = self.history[:self.position+1]
        self.history.append(site)
        self.position = len(self.history)-1

    def back(self, n):
        for i in range(n):
            if self.position == 0:
                break
            self.position -= 1
        return self.history[self.position]

    def forward(self, n):
        for i in range(n):
            if self.position >= len(self.history)-1:
                break
            self.position += 1
        return self.history[self.position]

# Input:
# ["BrowserHistory","visit","visit","visit","back","back","forward","visit","forward","back","back"]
# [["leetcode.com"],["google.com"],["facebook.com"],["youtube.com"],[1],[1],[1],["linkedin.com"],[2],[2],[7]]
# Output:
# [null,null,null,null,"facebook.com","google.com","facebook.com",null,"linkedin.com","google.com","leetcode.com"]

# L -> G -> F -> Y 
# L -> G -> F -> LIN
browserHistory = BrowserHistory("leetcode.com")
browserHistory.visit("google.com")                # // You are in "leetcode.com". Visit "google.com"
browserHistory.visit("facebook.com")              # // You are in "google.com". Visit "facebook.com"
browserHistory.visit("youtube.com")               # // You are in "facebook.com". Visit "youtube.com"
assert(browserHistory.back(1) == "facebook.com")  # // You are in "youtube.com", move back to "facebook.com" return "facebook.com"
assert(browserHistory.back(1) == "google.com")    # // You are in "facebook.com", move back to "google.com" return "google.com"
assert(browserHistory.forward(1) == "facebook.com") # // You are in "google.com", move forward to "facebook.com" return "facebook.com"
browserHistory.visit("linkedin.com")              # // You are in "facebook.com". Visit "linkedin.com"
assert(browserHistory.forward(2) == "linkedin.com") # // You are in "linkedin.com", you cannot move forward any steps.
assert(browserHistory.back(2) == "google.com")    # // You are in "linkedin.com", move back two steps to "facebook.com" then to "google.com". return "google.com"
assert(browserHistory.back(7) == "leetcode.com")  # // You are in "google.com", you can move back only one step to "leetcode.com". return "leetcode.com"